In [1]:
from src.pre_procesamiento.transformar_datos_para_modelos import (
  filtrar_datos_train_test_para_modelos,
  VARS_NUM,
  VARS_CATEGORICAS
)

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [2]:
datos_train, datos_test = filtrar_datos_train_test_para_modelos()
datos = pd.concat([datos_train, datos_test], ignore_index = True)
datos

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45217,33,Private,245211,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,<=50K
45218,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
45219,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
45220,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


In [3]:
"""
Para el modelo KNN, las variables numéricas deben estandarizarse
para que sus valores se encuentren entre 0 y 1 .

Asimismo, se transfomarán las variables categóricas vía
one-hot encoding, pero no es necesario descartar las 
columnas "extra" creadas .
"""
min_max_scaler = MinMaxScaler()
datos[VARS_NUM] = min_max_scaler.fit_transform(datos[VARS_NUM])

datos_encoded = pd.get_dummies(
  datos, 
  columns = VARS_CATEGORICAS,
  drop_first = False
)
datos_encoded

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,...,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,class_<=50K,class_>50K
0,0.301370,0.043350,0.800000,0.021740,0.0,0.397959,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
1,0.452055,0.047274,0.800000,0.000000,0.0,0.122449,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
2,0.287671,0.136877,0.533333,0.000000,0.0,0.397959,False,False,True,False,...,False,False,False,False,False,True,False,False,True,False
3,0.493151,0.149792,0.400000,0.000000,0.0,0.397959,False,False,True,False,...,False,False,False,False,False,True,False,False,True,False
4,0.150685,0.219998,0.800000,0.000000,0.0,0.397959,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45217,0.219178,0.156895,0.800000,0.000000,0.0,0.397959,False,False,True,False,...,False,False,False,False,False,True,False,False,True,False
45218,0.301370,0.136723,0.800000,0.000000,0.0,0.357143,False,False,True,False,...,False,False,False,False,False,True,False,False,True,False
45219,0.287671,0.244762,0.800000,0.000000,0.0,0.500000,False,False,True,False,...,False,False,False,False,False,True,False,False,True,False
45220,0.369863,0.047666,0.800000,0.054551,0.0,0.397959,False,False,True,False,...,False,False,False,False,False,True,False,False,True,False


In [4]:
datos_train_encoded = datos_encoded.iloc[0:datos_train.shape[0]]
datos_test_encoded = datos_encoded.iloc[datos_train.shape[0]:]

datos_train_encoded

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,...,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,class_<=50K,class_>50K
0,0.301370,0.043350,0.800000,0.021740,0.0,0.397959,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
1,0.452055,0.047274,0.800000,0.000000,0.0,0.122449,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
2,0.287671,0.136877,0.533333,0.000000,0.0,0.397959,False,False,True,False,...,False,False,False,False,False,True,False,False,True,False
3,0.493151,0.149792,0.400000,0.000000,0.0,0.397959,False,False,True,False,...,False,False,False,False,False,True,False,False,True,False
4,0.150685,0.219998,0.800000,0.000000,0.0,0.397959,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30157,0.136986,0.165081,0.733333,0.000000,0.0,0.377551,False,False,True,False,...,False,False,False,False,False,True,False,False,True,False
30158,0.315068,0.095390,0.533333,0.000000,0.0,0.397959,False,False,True,False,...,False,False,False,False,False,True,False,False,False,True
30159,0.561644,0.093721,0.533333,0.000000,0.0,0.397959,False,False,True,False,...,False,False,False,False,False,True,False,False,True,False
30160,0.068493,0.127292,0.533333,0.000000,0.0,0.193878,False,False,True,False,...,False,False,False,False,False,True,False,False,True,False


In [5]:
nombre_variable_target = 'class_>50K'

X_train, X_test, y_train, y_test = [
  datos_train_encoded.drop([nombre_variable_target], axis = 1).values,
  datos_test_encoded.drop([nombre_variable_target], axis = 1).values,
  datos_train_encoded[nombre_variable_target],
  datos_test_encoded[nombre_variable_target]
]

In [6]:
"""
Empleamos validación cruzada para estimar un candidato apropiado
del número de vecindades fijadas en el modelo KNN .
"""
cv = KFold(n_splits = 5, shuffle = True, random_state = 42)

resultados = {}
num_vecinos = [1, 3, 5, 7, 9]

for k in num_vecinos:
  knn = KNeighborsClassifier(n_neighbors = k)
  scores = cross_val_score(knn, X_train, y_train, cv = cv, scoring = 'accuracy')
  resultados[k] = {
    'exactitud_promedio': np.mean(scores),
    'desv_estandar': np.std(scores)
  }

for k, resultado in resultados.items():
  print(f'k={k}: Exactitud promedio = {resultado["exactitud_promedio"]:.4f}, Desviación estándar = {resultado["desv_estandar"]:.4f}')

k=1: Exactitud promedio = 0.9705, Desviación estándar = 0.0019
k=3: Exactitud promedio = 0.9714, Desviación estándar = 0.0015
k=5: Exactitud promedio = 0.9702, Desviación estándar = 0.0025
k=7: Exactitud promedio = 0.9681, Desviación estándar = 0.0023
k=9: Exactitud promedio = 0.9665, Desviación estándar = 0.0023


In [7]:
# Extraer para qué valor de K se obtuvo la mayor exactitud promedio
num_optimo_de_vecinos = num_vecinos[np.argmax([
  resultado['exactitud_promedio']
  for resultado in resultados.values()
])]

num_optimo_de_vecinos

3

In [8]:
modelo = KNeighborsClassifier(n_neighbors = num_optimo_de_vecinos).fit(X_train, y_train)
print(f'Exactitud del modelo: {round(100 * modelo.score(X_test, y_test), 2)}%')

# Matríz de confusión
confusion_matrix(y_test, modelo.predict(X_test))

Exactitud del modelo: 97.3%


array([[11252,   108],
       [  298,  3402]], dtype=int64)